In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install elasticsearch eng_to_ipa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 8.0 MB/s eta 0:00:00
  Created wheel for eng_to_ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822606 sha256=77c8507ba8b4b2c60e35c708ffccb389556383584cc7518664f6b17e86ebc70b
  Stored in directory: /root/.cache/pip/wheels/5b/ab/07/fe6722f710d8ef8bd0ccb4eb689ef96f5552f3fc0c80c1aa9c
Successfully built eng_to_ipa


In [3]:
! pip install faiss-gpu==1.7.2 sentence-transformers==2.2.2 numpy==1.25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=af98af19a77fc3bdcf115dd69a9ffab8d238fe6783bd5b22def20562e757b41b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [4]:
! pip install numpy==1.25

In [5]:
! pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=6f58a83c251bb37354f2282d9f434fd28737610f67802383a5a7b4e022f04a35
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [6]:
path = '../content/drive/MyDrive/JETFORK_11_2023/data/'

In [7]:
import faiss
import fasttext

In [8]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import re
from tqdm import tqdm
import sys

sys.path.insert(0, path)


In [9]:
es = Elasticsearch("http://5.35.83.33:9200/", basic_auth=("elastic", ""))

In [10]:
ENG_WORD_PATTERN = re.compile(r"[a-z]+")

In [11]:
model = fasttext.load_model(path + 'model/00_fasttext_queries.bin')

In [12]:
video_meta = pd.read_parquet(path + "clean_videos_done.parquet")[['video_id', 'video_title', 'channel_title',
       'v_channel_type', 'v_category', 'clean_title',
       'clean_channel_title']]

video_meta = video_meta.dropna(subset=['clean_title', 'clean_channel_title'])
video_meta.columns = ['id', 'source_video_title', 'source_channel_title', 'v_channel_type',
       'v_category', 'processed_video_title', 'processed_channel_title']
video_meta = video_meta.reset_index(drop=True)

In [13]:
import gc
gc.collect()

0

In [14]:
from tqdm import tqdm

tqdm.pandas()

In [15]:
video_meta = video_meta.head(15000000)

In [16]:
video_meta['video-vector'] = video_meta['processed_video_title'].progress_apply(lambda x: list(model.get_sentence_vector(x)))

100%|██████████| 15000000/15000000 [15:41<00:00, 15938.23it/s]


# Video vectors

In [17]:
INDEX_NAME = "videos_index__v1"
es.indices.create(index=INDEX_NAME,
                  body={
                      "mappings": {
                        "properties": {
                          "video-vector": {
                            "type": "dense_vector",
                            "dims": 70,
                            "similarity": "cosine"
                          },
                          "id": {"type": "text"},
                          "source_video_title": {"type": "text"},
                          "source_channel_title": {"type": "text"},
                          "v_channel_type": {"type": "text"},
                          "v_category": {"type": "text"},
                          "processed_video_title": {"type": "text"},
                          "processed_channel_title": {"type": "text"},
                          }
                      }
                    })

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'videos_index__v1'})

In [18]:
! pip install es_pandas

In [19]:
from es_pandas import es_pandas

In [20]:
ep = es_pandas("http://5.35.83.33:9200/")

In [ ]:
ep.to_es(video_meta, INDEX_NAME, thread_count=8, chunk_size=10000)

7310883660it [00:39, 371555526.40it/s]WARNING:elastic_transport.node_pool:Node <Urllib3HttpNode(http://5.35.83.33:9200)> has failed for 1 times in a row, putting on 1 second timeout
9412400206it [00:44, 436522320.90it/s]WARNING:elastic_transport.node_pool:Node <Urllib3HttpNode(http://5.35.83.33:9200)> has been marked alive after a successful request
9993798753it [00:46, 441629622.84it/s]WARNING:elastic_transport.node_pool:Node <Urllib3HttpNode(http://5.35.83.33:9200)> has failed for 1 times in a row, putting on 1 second timeout
11460752901it [00:49, 478015820.08it/s]WARNING:elastic_transport.node_pool:Node <Urllib3HttpNode(http://5.35.83.33:9200)> has failed for 2 times in a row, putting on 2 second timeout
13084847335it [00:52, 508052298.95it/s]WARNING:elastic_transport.node_pool:Node <Urllib3HttpNode(http://5.35.83.33:9200)> has failed for 3 times in a row, putting on 4 second timeout
15168029878it [00:56, 550887486.03it/s]WARNING:elastic_transport.node_pool:Node <Urllib3HttpNode(htt

In [ ]:
es.indices.refresh(index=INDEX_NAME)

ObjectApiResponse({'_shards': {'total': 100, 'successful': 50, 'failed': 0}})